<a href="https://colab.research.google.com/github/SridharanS2001/PIEL_model/blob/main/Codes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Model_Optimization**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb
import lightgbm as lgb
import warnings

d = pd.read_csv('/content/Final data.csv')
X = d.iloc[:, 0:11].values
y = d.iloc[:, 11].values

models = {
    'XGBoost': xgb.XGBRegressor(colsample_bytree=0.8, learning_rate=0.2, max_depth=3, n_estimators=100, subsample=1.0),
    'SVR': SVR(C=100, degree=2, epsilon=0.01, gamma='scale', kernel='rbf'),
    'RandomForest': RandomForestRegressor(max_depth=15, n_estimators=100, max_features='sqrt'),
    'DecisionTree': DecisionTreeRegressor(max_depth=15, min_samples_leaf=4),
    'LightGBM': lgb.LGBMRegressor(learning_rate=0.1, n_estimators=100, max_depth=-1, subsample=0.8, colsample_bytree=0.8),
    'Lasso': Lasso(alpha=0.0001),
    'KNN': KNeighborsRegressor(n_neighbors=11, p=1, weights='distance')
}


def evaluate_ensemble(X, y, model_names, k=5):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    val_rmse_scores, train_rmse_scores, r2_scores = [], [], []

    for train_idx, test_idx in kf.split(X):
        X_train, X_val = X[train_idx], X[test_idx]
        y_train, y_val = y[train_idx], y[test_idx]

        preds_train = np.zeros((len(y_train), len(model_names)))
        preds_val = np.zeros((len(y_val), len(model_names)))

        for i, name in enumerate(model_names):
            model = models[name]
            model.fit(X_train, y_train)
            preds_train[:, i] = model.predict(X_train)
            preds_val[:, i] = model.predict(X_val)

        y_pred_train = preds_train.mean(axis=1)
        y_pred_val = preds_val.mean(axis=1)

        train_rmse_scores.append(np.sqrt(mean_squared_error(y_train, y_pred_train)))
        val_rmse_scores.append(np.sqrt(mean_squared_error(y_val, y_pred_val)))
        r2_scores.append(r2_score(y_val, y_pred_val))

    return (np.mean(train_rmse_scores),
            np.mean(val_rmse_scores),
            np.mean(r2_scores))

def find_best_ensemble(X, y, min_models=2, max_models=7):
    results = []
    keys = list(models.keys())

    for r in range(min_models, min(max_models, len(keys)) + 1):
        for combo in combinations(keys, r):
            train_rmse, val_rmse, r2 = evaluate_ensemble(X, y, combo)
            results.append({
                'models': combo,
                'val_rmse': val_rmse,
                'train_rmse': train_rmse,
                'r2': r2
            })


    val_rmse_max = max(r['val_rmse'] for r in results)
    r2_max = max(r['r2'] for r in results)

    for r in results:
        norm_rmse = r['val_rmse'] / val_rmse_max
        norm_r2 = 1 - r['r2'] / r2_max
        r['score'] = norm_rmse + norm_r2

    results_sorted = sorted(results, key=lambda x: x['score'])

    return results_sorted


all_results = find_best_ensemble(X, y)


best = all_results[0]
print(" Best Model Combination:", best['models'])
print(f"   Validation RMSE: {best['val_rmse']:.4f}")
print(f"   Training RMSE: {best['train_rmse']:.4f}")
print(f"   R²: {best['r2']:.4f}")
print(f"   Combined Score: {best['score']:.4f}")


df_results = pd.DataFrame(all_results)
df_results['model_count'] = df_results['models'].apply(len)
df_results['model_names'] = df_results['models'].apply(lambda x: ' + '.join(x))


plt.figure(figsize=(14, 5))

plt.subplot(1, 3, 1)
plt.plot(df_results['model_names'], df_results['val_rmse'], 'o-', color='red', label='Val RMSE')
plt.xticks(rotation=90)
plt.title('Validation RMSE')
plt.ylabel('RMSE')
plt.grid(True)

plt.subplot(1, 3, 2)
plt.plot(df_results['model_names'], df_results['r2'], 'o-', color='blue', label='R²')
plt.xticks(rotation=90)
plt.title('Validation R²')
plt.ylabel('R² Score')
plt.grid(True)

plt.tight_layout()
plt.show()


print("Top 5 Model Combinations Based on Score:")
display(df_results[['model_names', 'val_rmse', 'r2', 'score']].sort_values(by='score').head(5))


### **Ensemble optimized Model**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
import lightgbm as lgb
import warnings


models = {

    'RandomForest': RandomForestRegressor(max_depth=15, n_estimators=100, max_features='sqrt', random_state=42),
    'DecisionTree': DecisionTreeRegressor(max_depth=15, min_samples_leaf=4, random_state=42),
    'LightGBM': lgb.LGBMRegressor(learning_rate=0.1, n_estimators=100, max_depth=-1, subsample=0.8, colsample_bytree=0.8, random_state=42),
}

kf = KFold(n_splits=5, shuffle=True, random_state=42)

rmse_train_list = []
r2_train_list = []
rmse_test_list = []
r2_test_list = []

for fold, (train_idx, test_idx) in enumerate(kf.split(X), 1):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    train_preds = []
    test_preds = []

    print(f"\n📂 Fold {fold}")

    for name, model in models.items():
        model.fit(X_train_scaled, y_train)
        train_pred = model.predict(X_train_scaled)
        test_pred = model.predict(X_test_scaled)

        train_preds.append(train_pred)
        test_preds.append(test_pred)

        print(f"    {name} - Train RMSE: {np.sqrt(mean_squared_error(y_train, train_pred)):.4f}, R²: {r2_score(y_train, train_pred):.4f}")
        print(f"    {name} - Test  RMSE: {np.sqrt(mean_squared_error(y_test, test_pred)):.4f}, R²: {r2_score(y_test, test_pred):.4f}")

    ensemble_train = np.mean(train_preds, axis=0)
    ensemble_test = np.mean(test_preds, axis=0)

    rmse_train = np.sqrt(mean_squared_error(y_train, ensemble_train))
    r2_train = r2_score(y_train, ensemble_train)
    rmse_test = np.sqrt(mean_squared_error(y_test, ensemble_test))
    r2_test = r2_score(y_test, ensemble_test)

    rmse_train_list.append(rmse_train)
    r2_train_list.append(r2_train)
    rmse_test_list.append(rmse_test)
    r2_test_list.append(r2_test)

    print(f" Ensemble - Train RMSE: {rmse_train:.4f}, R²: {r2_train:.4f}")
    print(f" Ensemble - Test  RMSE: {rmse_test:.4f}, R²: {r2_test:.4f}")


print(" Final Cross-Validation Results (Average of 5 folds):")
print(f" Train RMSE: {np.mean(rmse_train_list):.4f}, R²: {np.mean(r2_train_list):.4f}")
print(f" Test  RMSE: {np.mean(rmse_test_list):.4f}, R²: {np.mean(r2_test_list):.4f}")


### **Input_Parameter_calculation**

In [ ]:
import numpy as np
import pandas as pd
import metallurgy as mg


composition = {'Ti': 29.7 ,'Ni':50.3,'Ta': 4, 'Zr': 16}


alloy_str = ''.join([f"{elem}{amt}" for elem, amt in composition.items()])
alloy = mg.Alloy(alloy_str)


electronegativity = {'Ni': 1.91, 'Ti': 1.54, 'Si': 1.9, 'Co': 1.88, 'Mo': 2.16, 'Fe': 7.87, 'V': 1.63,
    'Cr': 1.66, 'Al': 1.61, 'Nb': 1.6, 'Mn': 1.55, 'Ta': 1.5, 'Cu': 1.9, 'Zr': 1.33, 'Hf': 1.3}
density = {'Ni': 8.91,'Hf': 13.3, 'Ti': 4.5, 'Si': 2.32, 'Co': 8.86, 'Mo': 10.2, 'Fe': 7.87, 'V': 6,
    'Cr': 7.15, 'Al': 2.7, 'Nb': 8.59, 'Mn': 7.3, 'Ta': 16.4, 'Cu': 8.9, 'Zr': 6.32}
radius = {'Hf': 212, 'Ni': 163, 'Ti': 187, 'Si': 210, 'Co': 192, 'Mo': 209, 'Fe': 194, 'V': 179,
    'Cr': 189, 'Al': 184, 'Nb': 207, 'Mn': 197, 'Ta': 217, 'Cu': 140, 'Zr': 186}
atomic_weight = { 'Hf': 178.49, 'Ni': 58.69, 'Ti': 48.867, 'Si': 28.085, 'Co': 58.933, 'Mo': 95.95, 'Fe': 55.84, 'V': 50.94,
    'Cr': 51.996, 'Al': 26.98, 'Nb': 92.90, 'Mn': 54.93, 'Ta': 180.94, 'Cu': 63.55, 'Zr': 91.22}
ie = {'Hf': 6.825, 'Ni': 7.64, 'Ti': 6.828, 'Si': 8.152, 'Co': 7.881, 'Mo': 7.092, 'Fe': 7.902, 'V': 6.746,
    'Cr': 6.767, 'Al': 5.986, 'Nb': 6.759, 'Mn': 7.434, 'Ta': 7.89, 'Cu': 7.726, 'Zr': 6.634}
cp = {'Hf': 144, 'Ni': 445, 'Ti': 520, 'Si': 710, 'Co': 421, 'Mo': 251, 'Fe': 449, 'V': 489,
    'Cr': 448, 'Al': 904, 'Nb': 265, 'Mn': 479, 'Ta': 140, 'Cu': 384.4, 'Zr': 278}
E = {'Hf': 4, 'Ni': 10, 'Ti': 4, 'Si': 4, 'Co': 9, 'Mo': 6, 'Fe': 8, 'V': 5,
    'Cr': 6, 'Al': 3, 'Nb': 5, 'Mn': 7, 'Ta': 5, 'Cu': 11, 'Zr': 4}


def mean_electronegativity(comp):
    total = sum(comp[e] for e in comp if e in electronegativity)
    weighted_sum = sum(comp[e] * electronegativity[e] for e in comp if e in electronegativity)
    return weighted_sum / total if total != 0 else 0

def custom_dX(comp):
    weighted_total = mean_electronegativity(comp)
    terms = [
        float(comp[e]) * ((weighted_total - electronegativity[e]) ** 2)
        for e in comp if e in electronegativity
    ]
    return np.sqrt(sum(terms))

def mean_ie(comp): return sum(float(comp[e]) * ie[e] for e in comp)/100
def mean_E(comp): return sum(float(comp[e]) * E[e] for e in comp)/100
def mean_mass(comp): return sum(float(comp[e]) * atomic_weight[e] for e in comp)/100
def mean_density(comp): return sum(float(comp[e]) * density[e] for e in comp)/100

def xmean_density(comp):
    total = sum(comp[e] for e in comp if e in density)
    weighted_sum = sum(comp[e] * density[e] for e in comp if e in density)
    return weighted_sum / total if total != 0 else 0
def custom_dd(comp):
    weighted_total = xmean_density(comp)
    terms = [
        float(comp[e]) * ((weighted_total - density[e]) ** 2)
        for e in comp if e in density
    ]
    return np.sqrt(sum(terms))

def mean_radius(comp): return sum(float(comp[e]) * radius[e] for e in comp)/100
def mean_cp(comp): return sum(float(comp[e]) * cp[e] for e in comp)/100


def mie(comp):
    total = sum(comp[e] for e in comp if e in density)
    weighted_sum = sum(comp[e] * density[e] for e in comp if e in density)
    return weighted_sum / total if total != 0 else 0
def die(comp):
    weighted_total = mie(comp)
    terms = [
        float(comp[e]) * ((weighted_total - ie[e]) ** 2)
        for e in comp if e in ie
    ]
    return np.sqrt(sum(terms))

def enthalpy(comp):
    hmat = pd.read_csv('/content/Enthalpy_Mixing1_filled.csv', index_col=0)
    elements = list(comp.keys())
    h = 0
    for i in range(len(elements)):
        for j in range(i+1, len(elements)):
            Ci = float(comp[elements[i]]) / 100
            Cj = float(comp[elements[j]]) / 100
            Hij = hmat.loc[elements[i], elements[j]]
            h += 4 * Ci * Cj * Hij
    return h

print("Ideal Entropy (J/mol·K):", mg.entropy.ideal_entropy(alloy))
print("Mean Ionization Energy (eV):", mean_ie(composition))
print("Mean Atomic Mass (g/mol):", mean_mass(composition))
print("Mean Density (g/cm³):", mean_density(composition))
print("Electronegativity Difference (dX):", custom_dX(composition))
print("Density Deviation:", custom_dd(composition))
print("Mean Atomic Radius (pm):", mean_radius(composition))
print("Mean Heat Capacity (J/kg·K):", mean_cp(composition))
print("Mean Valence Electron Concentration (e/a):", mean_E(composition))
print("IE Deviation:", die(composition))
print("Enthalpy of Mixing (kJ/mol):", enthalpy(composition))

### **Prediction**

In [ ]:
new_input = pd.DataFrame([[1.1281769895955966,
                            7.247875999999999,
                            65.867369,
                            7.48543,
                            2.306661481448892,
                            26.5781602356145,
                            175.968,
                            428.355,
                            5.127588122499698,
                            -38.78921088,
                            0]])

new_input_scaled = scaler.transform(new_input)


new_preds = [model.predict(new_input_scaled)[0] for model in models.values()]

ensemble_prediction = np.mean(new_preds)
c = ensemble_prediction - 273

print(" Individual Model Predictions:")
for name, pred in zip(models.keys(), new_preds):
    print(f"    {name}: {pred:.4f}")

print(f" Ensemble Prediction in K: {ensemble_prediction:.4f}")
print(f" Ensemble Prediction in C: {c:.4f}")

### **SHAP plot**

In [ ]:
import shap

explainer_rf = shap.Explainer(models['RandomForest'], X_train_scaled)
explainer_dt = shap.Explainer(models['DecisionTree'], X_train_scaled)
explainer_lgb = shap.Explainer(models['LightGBM'], X_train_scaled)

shap_values_rf = explainer_rf(X_test_scaled)
shap_values_dt = explainer_dt(X_test_scaled)
shap_values_lgb = explainer_lgb(X_test_scaled)

avg_shap_values = (shap_values_rf.values + shap_values_dt.values + shap_values_lgb.values) / 3

shap_values_ensemble = shap.Explanation(
    values=avg_shap_values,
    base_values=np.mean([shap_values_rf.base_values, shap_values_dt.base_values, shap_values_lgb.base_values], axis=0),
    data=X_test_scaled,
    feature_names=df.columns[:11].tolist()
)

shap.plots.beeswarm(shap_values_ensemble, max_display=10)


### **Correlation**

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import lightgbm as lgb

os.makedirs("Ms_vs_Features_CSV", exist_ok=True)


X = df.iloc[:, :11].values
y = df.iloc[:, 11].values
feature_names = df.columns[:11]


models = {
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
    'DecisionTree': DecisionTreeRegressor(max_depth=15, min_samples_leaf=4, random_state=42),
    'LightGBM': lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1, subsample=0.8,
                                   colsample_bytree=0.8, random_state=42),
}

predicted_features = []
target_range = np.linspace(y.min(), y.max(), 100)

for i in range(X.shape[1]):
    predictions = []

    for model_name, model in models.items():
        model.fit(y.reshape(-1, 1), X[:, i])
        pred = model.predict(target_range.reshape(-1, 1))
        predictions.append(pred)

    ensemble_pred = np.mean(predictions, axis=0)
    predicted_features.append(ensemble_pred)

    df_out = pd.DataFrame({
        'Ms_Temperature': target_range,
        f'Ensemble_Predicted_{feature_names[i]}': ensemble_pred
    })
    csv_filename = f'Ms_vs_Features_CSV/Ms_vs_{feature_names[i]}.csv'
    df_out.to_csv(csv_filename, index=False)

    plt.figure()
    plt.plot(target_range, ensemble_pred, label=f'Ensemble Predicted {feature_names[i]}', color='darkorange')
    plt.xlabel('Ms Temperature')
    plt.ylabel(f'{feature_names[i]}')
    plt.title(f'Ms vs {feature_names[i]} (Ensemble Prediction)')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()


### **PCA_based classification**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier

df = pd.read_csv('/content/Final data.csv')
X = df.iloc[:, :11].values
y = df.iloc[:, 11].values


bins = [0, 973, float('inf')]
labels = [1, 2]
df['Ms_Category'] = pd.cut(y, bins=bins, labels=labels, right=False)


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


pca = PCA(n_components=2)
X_2D = pca.fit_transform(X_scaled)


x_min, x_max = X_2D[:, 0].min() - 1, X_2D[:, 0].max() + 1
y_min, y_max = X_2D[:, 1].min() - 1, X_2D[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))


knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_2D, df['Ms_Category'])

Z = knn.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)


plt.figure(figsize=(8, 6))


plt.contourf(xx, yy, Z, alpha=0.4, cmap="coolwarm")

sns.scatterplot(x=X_2D[:, 0], y=X_2D[:, 1], hue=df['Ms_Category'], palette="coolwarm", s=50, edgecolor='k')


plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title('Contour Plot of Ms Categories (2 classes)')
plt.legend(title='Ms Category')
plt.grid(True)
plt.tight_layout()
plt.show()
